# **DeepLearning Project**

In [2]:
import zipfile
import os

# Paths
zip_file_path = "/content/Student_Latest_Data.zip"
extract_to_path = "/content/Student_Latest_Data"

# Unzip the file
os.makedirs(extract_to_path, exist_ok=True)  # Ensure the extraction folder exists
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

# Verify extracted files
extracted_files = os.listdir(extract_to_path)
extracted_files


['Student_Latest_Data']

['Student_Latest_Data']

In [3]:
import pandas as pd

In [4]:
# Path to the Excel file
excel_file_path = "/content/Student_Latest_Data/Student_Latest_Data/student_data.xlsx"

# Read the Excel file
student_data = pd.read_excel(excel_file_path)

# Display the first few rows of the data
student_data.head()


,Roll No,First Name,Middle Name,Last Name,Degree 1,Degree 1 Primary Specializations,Projects Name,Projects Key Skills,Internship Experience - Organization Name (Industry - Stipend (In INR) - Start date to End date[Duration]) - Mentor Name (Designation - Phone - Email) - Academic Guide Name 1,Internship Experience - Organization Name (Industry - Stipend (In INR) - Start date to End date[Duration]) - Mentor Name (Designation - Phone - Email) - Academic Guide Name 2,...,Co-Curricular Activities,Extra Curricular Activities,Seminar/Trainings/Workshops Title,Seminar/Trainings/Workshops Key Skills,Other Professional Experiences - Organization Name (Designation - Department - CTC (In INR) - Start date to End date[Duration]) 1,Other Professional Experiences - Organization Name (Designation - Department - CTC (In INR) - Start date to End date[Duration]) 2,Remaining Other Professional Experiences - Organization Name (Designation - Department - CTC (In INR) - Start date to End date[Duration]),Other Professional Experience - Industry,Other Professional Experiences - Key Skills,Assessments/Certification (Name - Aggregate Marks)
0,23070243001,Adrija,NaN,Paul,M.Sc.,Data Science,Three Demographic Issues- Population Projectio...,Tableau; LLM; Multiagent framework; R programm...,Technocolabs Softwares (Banking / Financial Se...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Business Intelligence Professional Cert...
1,23070243002,Anviya,Mariam,Bobby,M.Sc.,Data Science,Text Recognition Using Tesseract & Open CV; Pe...,OpenCV; Natural Language Processing; Tesseract...,Prodigy Infotech (IT / Computers - Software - ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBM Big Data with Spark and Hadoop - 83.320;
2,23070243004,Ayush,NaN,Dutta,M.Sc.,Data Science,Investment Risk Management; NLP for ESG Analys...,Unsupervised Learning; ML; Data Analysis; Opti...,Indian Statistical Institute (NA - NA - 2024-0...,Opsis System Pvt Ltd (Academics / Research - N...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23070243005,Bhumika,NaN,Goyal,M.Sc.,Data Science,Salary Estimation using K-Nearest Neighbor; Fa...,Analysis; Python; analytical; Machine learning,Skysight.Ai (Technology - 5000 - 2024-03-16 to...,Mentorness (Technology - NA - 2024-02-01 to 20...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hacker Rank SQL(Basic) - NA; Chatgpt Explorati...
4,23070243006,Chorge,Nikita,Dinesh,M.Sc.,Data Science,Language detection and Translation; Customer s...,Handling data imbalance; Feature engineering; ...,Prodigy Infotech (NA - NA - 2024-02-01 to 2024...,NaN,...,SIG's Decoration Committee Member,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Introduction to Big Data with Spark and Hadoop...


In [5]:
# Path to the Excel file
excel_file_path = "/content/new_jobs_jd_300.csv"

# Read the Excel file
job_data = pd.read_csv(excel_file_path)

# Display the first few rows of the data
job_data.head()

,Job Title,Company,Location,Link,Job_Description
0,Data science interns,Zen Consultancy,"Mumbai, Maharashtra, India (On-site)",https://www.linkedin.com/jobs/view/4076706303/...,"Skills:\nPython programming, Statistical analy..."
1,Data Analyst Intern,FuelBuddy,"Gurgaon, Haryana, India (On-site)",https://www.linkedin.com/jobs/view/4076700680/...,"Skills:\nMicrosoft Power BI, Python, Advance E..."
2,Data Analytics Trainee,Ovid Metrics,"Nashik, Maharashtra, India (On-site)",https://www.linkedin.com/jobs/view/4076555143/...,"Skills:\nPython, Tableau, SQL,\n\nKey Skills\n..."
3,Data science interns,Zen consultancy,"Bengaluru East, Karnataka, India (On-site)",https://www.linkedin.com/jobs/view/4076709034/...,"Skills:\nPython programming, Statistical analy..."
4,Data Scientist,Web Secure AI,Mumbai Metropolitan Region (On-site),https://www.linkedin.com/jobs/view/4078149604/...,"Skills:\nenglish, computer litracy, data scien..."


In [6]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.7 MB/s eta 0:00:00


In [14]:
import os
import spacy
import pandas as pd
import PyPDF2
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Set up spaCy
nlp = spacy.load("en_core_web_sm")  # Download spaCy English model

# Paths
excel_path = "/content/Student_Latest_Data/Student_Latest_Data/student_data.xlsx"
resume_folder = "/content/Student_Latest_Data/Student_Latest_Data/New_resume"
job_csv_path = "/content/new_jobs_jd_300.csv"

# Load the job data
job_data = pd.read_csv(job_csv_path)

# Ensure required columns are present
required_columns = ['Job Title', 'Company', 'Location', 'Link', 'Job_Description']
if not all(col in job_data.columns for col in required_columns):
    raise ValueError("Job data CSV file is missing required columns.")

# Function to preprocess text using spaCy (removes stop words and punctuation)
def preprocess_text(text):
    doc = nlp(text.lower())  # Process the text using spaCy
    filtered_words = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return " ".join(filtered_words)

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
            return text
    except Exception as e:
        return f"Error reading PDF: {e}"

# Function to recommend jobs
def recommend_jobs(resume_text, job_data, top_n=5):
    # Preprocess job descriptions
    job_data['Processed_Job_Description'] = job_data['Job_Description'].fillna("").apply(preprocess_text)

    # Encode job descriptions and resume using Sentence-BERT
    model = SentenceTransformer('all-MiniLM-L6-v2')
    job_embeddings = model.encode(job_data['Processed_Job_Description'].tolist(), convert_to_tensor=True)
    resume_embedding = model.encode(preprocess_text(resume_text), convert_to_tensor=True)

    # Move tensors to CPU and convert to NumPy arrays
    job_embeddings = job_embeddings.cpu().numpy()
    resume_embedding = resume_embedding.cpu().numpy()

    # Compute cosine similarity
    similarity_scores = cosine_similarity([resume_embedding], job_embeddings).flatten()
    job_data['similarity_score'] = similarity_scores

    # Get top job recommendations
    top_jobs = job_data.nlargest(top_n, 'similarity_score')[['Job Title', 'Company', 'Location', 'Link', 'similarity_score']]
    return top_jobs

# Main function for user input
def main():
    # User input
    roll_no = input("Enter the Roll Number: ").strip()
    resume_path = os.path.join(resume_folder, f"{roll_no}.pdf")

    if not os.path.exists(resume_path):
        print(f"Resume not found for Roll No: {roll_no}")
        return

    # Extract text from the resume
    resume_text = extract_text_from_pdf(resume_path)
    if resume_text.startswith("Error"):
        print(f"Error reading resume for Roll No: {roll_no}")
        return

    # Recommend jobs
    top_jobs = recommend_jobs(resume_text, job_data)

    # Display top 5 jobs
    print(f"\nTop 5 Job Recommendations for Roll No: {roll_no}")
    for idx, job in enumerate(top_jobs.to_dict(orient='records'), start=1):
        print(f"\nJob {idx}:")
        print(f"  - Title: {job['Job Title']}")
        print(f"  - Company: {job['Company']}")
        print(f"  - Location: {job['Location']}")
        print(f"  - Similarity Score: {job['similarity_score']:.2f}")
        print(f"  - [Apply Here]({job['Link']})")

# Run the main function
if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Enter the Roll Number: 23070243065

Top 5 Job Recommendations for Roll No: 23070243065

Job 1:
  - Title: Data Science
  - Company: NextGen AI solutions
  - Location: Hyderabad, Telangana, India (On-site)
  - Similarity Score: 0.84
  - [Apply Here](https://www.linkedin.com/jobs/view/4076708831/?eBP=CwEAAAGTRaHGdIfOi4YnNHs_SCX8qerjjy522hazv2ytHwY-1pQYkq6ysmVBIgpyJsDB6mBPODC_4qUPE59rFwqNr1l1zgOJNNNnq7saoAFPWdeAYez01lHK5XRJu2quQe-SWQpgQUqcMxBpmyYSXZ3ynUq2jKtBvI8CPA9DYoS29FZViQUhgT2PSxPcqQaBcl8sl7x1d76AXPnL5BJTvg19Dsu3DK69D8YGEh-fX7grgmBQWm-6jpOkmUNznk048lvMctoU_AHNor7wXGYY3DxeR5iXEt7OPEPpphkQvVecb_etTRrnDghWq7RzAFivu7WVLrLPTd_5gns-NiAiYR5Gqdq4DizxgTSkqVBuuDQl1hLka0lRnqj8xo93EEPfPmx9t40ShubskUrYLJbokD-I7Hhfy7qwfeYRLsVCd2p25H-xmaH1Bwlo5rGLNJxIkcDbIxJH5YSt63S_YwCDCba6AeGujQhedPZpoYam5ooT4cimj96XY5VIzAHz-D-Fx0Y&refId=3azH6MS4jVtUXR%2BP5fZ01w%3D%3D&trackingId=JUWtsH%2BPE4WYXo3MOvvUaQ%3D%3D&trk=flagship3_search_srp_jobs)

Job 2:
  - Title: Data Scientist
  - Company: Web Secure AI
  - Location:

# **Interface**

In [10]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [15]:
import os
import pandas as pd
import PyPDF2
import gradio as gr
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Paths
excel_path = "/content/Student_Latest_Data/Student_Latest_Data/student_data.xlsx"
resume_folder = "/content/Student_Latest_Data/Student_Latest_Data/New_resume"
job_csv_path = "/content/new_jobs_jd_300.csv"

import os
import spacy
import pandas as pd
import PyPDF2
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr

# Set up spaCy
nlp = spacy.load("en_core_web_sm")  # Ensure the spaCy English model is downloaded

# Load the job data
job_data = pd.read_csv(job_csv_path)

# Ensure required columns are present
required_columns = ['Job Title', 'Company', 'Location', 'Link', 'Job_Description']
if not all(col in job_data.columns for col in required_columns):
    raise ValueError("Job data CSV file is missing required columns.")

# Function to preprocess text using spaCy (removes stop words and punctuation)
def preprocess_text(text):
    doc = nlp(text.lower())  # Process the text using spaCy
    filtered_words = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return " ".join(filtered_words)

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
            return text
    except Exception as e:
        return f"Error reading PDF: {e}"

# Function to recommend jobs
def recommend_jobs(resume_text, job_data, top_n=5):
    # Preprocess job descriptions
    job_data['Processed_Job_Description'] = job_data['Job_Description'].fillna("").apply(preprocess_text)

    # Encode job descriptions and resume using Sentence-BERT
    model = SentenceTransformer('all-MiniLM-L6-v2')
    job_embeddings = model.encode(job_data['Processed_Job_Description'].tolist(), convert_to_tensor=True)
    resume_embedding = model.encode(preprocess_text(resume_text), convert_to_tensor=True)

    # Move tensors to CPU and convert to NumPy arrays
    job_embeddings = job_embeddings.cpu().numpy()
    resume_embedding = resume_embedding.cpu().numpy()

    # Compute cosine similarity
    similarity_scores = cosine_similarity([resume_embedding], job_embeddings).flatten()
    job_data['similarity_score'] = similarity_scores

    # Get top job recommendations
    top_jobs = job_data.nlargest(top_n, 'similarity_score')[['Job Title', 'Company', 'Location', 'Link', 'similarity_score']]
    return top_jobs

# Gradio function
def job_recommender(roll_no):
    resume_path = os.path.join(resume_folder, f"{roll_no}.pdf")

    if not os.path.exists(resume_path):
        return f"Resume not found for Roll No: {roll_no}"

    # Extract text from the resume
    resume_text = extract_text_from_pdf(resume_path)
    if resume_text.startswith("Error"):
        return f"Error reading resume for Roll No: {roll_no}"

    # Recommend jobs
    top_jobs = recommend_jobs(resume_text, job_data)

    # Prepare a response
    results = []
    for idx, job in enumerate(top_jobs.to_dict(orient='records'), start=1):
        job_info = f"**Job {idx}:**\n" \
                   f"- **Title:** {job['Job Title']}\n" \
                   f"- **Company:** {job['Company']}\n" \
                   f"- **Location:** {job['Location']}\n" \
                   f"- **Similarity Score:** {job['similarity_score']:.2f}\n" \
                   f"- [Apply Here]({job['Link']})"
        results.append(job_info)

    return "\n\n".join(results)

# Set up Gradio interface
iface = gr.Interface(
    fn=job_recommender,
    inputs=gr.Textbox(label="Enter Roll Number"),
    outputs=gr.Markdown(label="Job Recommendations"),
    title="Job Recommendation System",
    description="Enter your roll number to get personalized job recommendations based on your resume."
)

# Launch the app
if __name__ == "__main__":
    iface.launch()


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f402b45ca1a0cc6ec8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **InterFace 2**

In [18]:
import os
import spacy
import pandas as pd
import PyPDF2
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr

# Set up spaCy
nlp = spacy.load("en_core_web_sm")  # Ensure the spaCy English model is downloaded

# Paths
excel_path = "/content/Student_Latest_Data/Student_Latest_Data/student_data.xlsx"
resume_folder = "/content/Student_Latest_Data/Student_Latest_Data/New_resume"
job_csv_path = "/content/new_jobs_jd_300.csv"


# Load the student data (Roll Number and First Name)
student_data = pd.read_excel(excel_path, usecols=["Roll No", "First Name"])

# Normalize the Roll Numbers (convert to string and strip spaces)
student_data['Roll No'] = student_data['Roll No'].astype(str).str.strip()
student_data.set_index("Roll No", inplace=True)

# Load the job data
job_data = pd.read_csv(job_csv_path)

# Ensure required columns are present
required_columns = ['Job Title', 'Company', 'Location', 'Link', 'Job_Description']
if not all(col in job_data.columns for col in required_columns):
    raise ValueError("Job data CSV file is missing required columns.")

# Function to preprocess text using spaCy (removes stop words and punctuation)
def preprocess_text(text):
    doc = nlp(text.lower())  # Process the text using spaCy
    filtered_words = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return " ".join(filtered_words)

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
            return text
    except Exception as e:
        return f"Error reading PDF: {e}"

# Function to recommend jobs
def recommend_jobs(resume_text, job_data, top_n=5):
    # Preprocess job descriptions
    job_data['Processed_Job_Description'] = job_data['Job_Description'].fillna("").apply(preprocess_text)

    # Encode job descriptions and resume using Sentence-BERT
    model = SentenceTransformer('all-MiniLM-L6-v2')
    job_embeddings = model.encode(job_data['Processed_Job_Description'].tolist(), convert_to_tensor=True)
    resume_embedding = model.encode(preprocess_text(resume_text), convert_to_tensor=True)

    # Move tensors to CPU and convert to NumPy arrays
    job_embeddings = job_embeddings.cpu().numpy()
    resume_embedding = resume_embedding.cpu().numpy()

    # Compute cosine similarity
    similarity_scores = cosine_similarity([resume_embedding], job_embeddings).flatten()
    job_data['similarity_score'] = similarity_scores

    # Get top job recommendations
    top_jobs = job_data.nlargest(top_n, 'similarity_score')[['Job Title', 'Company', 'Location', 'Link', 'similarity_score']]
    return top_jobs

# Gradio function
def job_recommender(roll_no):
    roll_no = str(roll_no).strip()  # Normalize input roll number
    if roll_no not in student_data.index:
        return f"Roll Number {roll_no} not found in student data.", ""

    first_name = student_data.loc[roll_no, "First Name"]
    resume_path = os.path.join(resume_folder, f"{roll_no}.pdf")

    if not os.path.exists(resume_path):
        return first_name, f"Resume not found for Roll No: {roll_no} ({first_name})"

    # Extract text from the resume
    resume_text = extract_text_from_pdf(resume_path)
    if resume_text.startswith("Error"):
        return first_name, f"Error reading resume for Roll No: {roll_no} ({first_name})"

    # Recommend jobs
    top_jobs = recommend_jobs(resume_text, job_data)

    # Prepare a response
    recommendations = f"**Top Job Recommendations for {first_name} (Roll No: {roll_no}):**\n\n"
    for idx, job in enumerate(top_jobs.to_dict(orient='records'), start=1):
        job_info = f"**Job {idx}:**\n" \
                   f"- **Title:** {job['Job Title']}\n" \
                   f"- **Company:** {job['Company']}\n" \
                   f"- **Location:** {job['Location']}\n" \
                   f"- **Similarity Score:** {job['similarity_score']:.2f}\n" \
                   f"- [Apply Here]({job['Link']})"
        recommendations += job_info + "\n\n"

    return first_name, recommendations

# Set up Gradio interface
iface = gr.Interface(
    fn=job_recommender,
    inputs=gr.Textbox(label="Enter Roll Number"),
    outputs=[
        gr.Textbox(label="First Name"),
        gr.Markdown(label="Job Recommendations")
    ],
    title="Job Recommendation System",
    description="Enter your roll number to get your name and personalized job recommendations based on your resume."
)

# Launch the app
if __name__ == "__main__":
    iface.launch()


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dfd7fff9a60c5a07ef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
